# P7 (Соціальне забезпечення та соціальний захист населення): 1/5

In [1]:
import pandas as pd
from src.utils import normalize_parameter, save_data
from src.utils import PATH_INTERIM

## p7_01
### кількість витрачених коштів на придбання технічних та інших засобів реабілітації у розрахунку на одну особу з інвалідністю

In [2]:
df_P07_002 = pd.read_excel(PATH_INTERIM / "P7" / "P07_002.xlsx")
df_P07_002["p7_01_raw"] = (
    df_P07_002["Кількість технічних засобів реабілітації"] / 
    df_P07_002["Кількість осіб"]
)
df_P07_002["p7_01"] = normalize_parameter(df_P07_002["p7_01_raw"])

feature_range=(0, 1); fill_na=True; array_bounds=(0.0, 0.024266365688487584); normalization_bounds=(0.0, 0.024266365688487584)
